# Practica 2 - Natural Language processing

In [34]:
import numpy as np

import tensorflow as tf

import pandas as pd
from collections import Counter
import re

from tensorflow import keras


import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import Model, Input, layers
from tensorflow.keras.layers import Embedding, Dot, Reshape, Dense

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams

from keras.callbacks import TensorBoard

import re
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer

import numpy as np
from tensorflow.keras.preprocessing.sequence import skipgrams

In [9]:
import gdown
import os

# Crear el directorio de datos si no existe
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# URL de Google Drive en formato correcto para gdown
url = "https://drive.google.com/uc?id=1GXSUzaXDvrlcimgTwRR9tmDr6xr8gu49"
zip_filename = "sentiment_analysis.zip"
zip_path = os.path.join(data_dir, zip_filename)

# Descargar el archivo
if not os.path.exists(zip_path):
    gdown.download(url, zip_path, quiet=False)
else:
    print("Data zipfile already exists")

Downloading...
From: https://drive.google.com/uc?id=1GXSUzaXDvrlcimgTwRR9tmDr6xr8gu49
To: /content/data/sentiment_analysis.zip
100%|██████████| 1.86M/1.86M [00:00<00:00, 185MB/s]


In [10]:
import shutil
from zipfile import ZipFile
from concurrent.futures import ThreadPoolExecutor

data_dir = "data"
zip_filename = "sentiment_analysis.zip"
zip_path = os.path.join(data_dir, zip_filename)
files = ["train.csv", "test.csv"]
full_paths = [os.path.join(data_dir, file) for file in files]

if not all(os.path.isfile(path) for path in full_paths):
    with ZipFile(zip_path, 'r') as zf:
        with ThreadPoolExecutor() as exe:
            for file in zf.namelist():
                if not file.startswith("__MACOSX"):
                    exe.submit(zf.extract, file, path=data_dir)
else:
    print("test and train files already exist")

In [35]:
import os
import pandas as pd

data_dir = "data"
train_path = os.path.join(data_dir, "train.csv")
test_path = os.path.join(data_dir, "test.csv")
train_df = pd.read_csv(train_path, encoding='ISO-8859-1', index_col="textID")
test_df = pd.read_csv(test_path, encoding='ISO-8859-1', index_col="textID")
train_df.head()

,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
textID,,,,,,,,,
cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [36]:
from collections import Counter
import re
# 5. Juntar los DataFrames para facilitar el preprocesado
data_df = pd.concat([train_df, test_df], ignore_index=True)      # concatenación[2]

# 6. Inspeccionar estructura básica
print(data_df.info())
print(data_df.head())

# 7. Contar palabras únicas en la columna 'text' para estimar VOCAB_SIZE
all_text = ' '.join(data_df['text'].astype(str)).lower()
words = re.findall(r'\b\w+\b', all_text)
word_counts = Counter(words)
unique_words = len(word_counts)

print(f"Total de muestras: {data_df.shape[0]}")
print(f"Palabras únicas encontradas: {unique_words}")
print("Top 10 palabras más frecuentes:", word_counts.most_common(10))

# 8. Definir parámetros para TensorFlow/Keras
BUFFER_SIZE = 32768      # chivo mayor al dataset para buen shuffle 2^15
BATCH_SIZE = 128        # potencia de 2 adecuada para GPU
VOCAB_SIZE = 8192      # tamaño del vocabulario[4]

print("BUFFER_SIZE =", BUFFER_SIZE)
print("BATCH_SIZE =", BATCH_SIZE)
print("VOCAB_SIZE =", VOCAB_SIZE)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32296 entries, 0 to 32295
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   text              31014 non-null  object 
 1   selected_text     27480 non-null  object 
 2   sentiment         31015 non-null  object 
 3   Time of Tweet     31015 non-null  object 
 4   Age of User       31015 non-null  object 
 5   Country           31015 non-null  object 
 6   Population -2020  31015 non-null  float64
 7   Land Area (Km²)   31015 non-null  float64
 8   Density (P/Km²)   31015 non-null  float64
dtypes: float64(3), object(6)
memory usage: 2.2+ MB
None
                                                text  \
0                I`d have responded, if I were going   
1      Sooo SAD I will miss you here in San Diego!!!   
2                          my boss is bullying me...   
3                     what interview! leave me alone   
4   Sons of ****, why couldn`t they pu

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer
# 1. Seleccionar las 4 096 palabras más frecuentes
most_common_words = {w for w, _ in word_counts.most_common(VOCAB_SIZE)}

# 2. Filtrar cada texto para quedarnos solo con tokens en el top 4 096
corpus_filtered = []
for text in data_df['text'].dropna().astype(str):
    tokens = re.findall(r'\b\w+\b', text.lower())
    filtered_tokens = [t for t in tokens if t in most_common_words]
    corpus_filtered.append(" ".join(filtered_tokens))

# 3. Tokenizar el corpus filtrado
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(corpus_filtered)

# 4. Convertir textos a secuencias de índices
sequences = tokenizer.texts_to_sequences(corpus_filtered)

# 5. Diccionarios de mapeo y tamaño final de vocabulario
word2idx = tokenizer.word_index
idx2word = {i: w for w, i in word2idx.items()}
vocab_size = min(len(word2idx) + 1, VOCAB_SIZE)

# 6. Mostrar resultados de prueba
print("Primeros 5 textos filtrados:", corpus_filtered[:5])
print("Primeras 5 secuencias:", sequences[:5])
print("Tamaño de vocabulario efectivo:", vocab_size)

Primeros 5 textos filtrados: ['i d have responded if i were going', 'sooo sad i will miss you here in san diego', 'my boss is me', 'what interview leave me alone', 'sons of why couldn t they put them on the releases we already bought']
Primeras 5 secuencias: [[1, 163, 19, 7648, 71, 1, 151, 49], [421, 117, 1, 63, 94, 7, 91, 10, 1447, 2230], [5, 1410, 9, 16], [51, 1193, 350, 16, 495], [4254, 13, 118, 472, 14, 72, 332, 131, 17, 3, 7649, 50, 210, 569]]
Tamaño de vocabulario efectivo: 8152


In [111]:
from tensorflow.keras.layers import TextVectorization

vectorize_layer = TextVectorization(
    output_mode='int',
    standardize="lower_and_strip_punctuation",
    split="whitespace",
)

# corpus = df_train["text"].values

vectorize_layer.adapt(corpus_filtered)

vectorized_train = vectorize_layer(corpus_filtered)
vectorized_train

<tf.Tensor: shape=(31014, 37), dtype=int64, numpy=
array([[   2,  164,   20, ...,    0,    0,    0],
       [ 423,  118,    2, ...,    0,    0,    0],
       [   6, 1444,   10, ...,    0,    0,    0],
       ...,
       [   2,   59,   52, ...,    0,    0,    0],
       [  52,   10,   43, ...,    0,    0,    0],
       [  42,  149,   60, ...,    0,    0,    0]])>

In [112]:
vectorize_layer.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('i'),
 np.str_('to'),
 np.str_('the'),
 np.str_('a'),
 np.str_('my'),
 np.str_('it'),
 np.str_('you'),
 np.str_('and')]

In [117]:
import tensorflow as tf
train_ds = tf.data.Dataset.from_tensor_slices(vectorized_train)

vocab_size = len(vectorize_layer.get_vocabulary())

print(train_ds.element_spec)
train_ds.as_numpy_iterator().next()

TensorSpec(shape=(37,), dtype=tf.int64, name=None)


array([   2,  164,   20, 7779,   72,    2,  152,   50,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0])

In [118]:
import tensorflow as tf

def generate_skipgram_pairs(sequence, window_size):
    seq_len = tf.shape(sequence)[0]
    positions = tf.range(seq_len)

    def extract_context(i):
        start = tf.maximum(0, i - window_size)
        end = tf.minimum(seq_len, i + window_size + 1)

        # Exclude the center word itself
        left = sequence[start:i]
        right = sequence[i + 1:end]
        context = tf.concat([left, right], axis=0)

        target = sequence[i]
        targets = tf.fill([tf.shape(context)[0]], target)

        return tf.data.Dataset.from_tensor_slices((targets, context))

    return tf.data.Dataset.from_tensor_slices(positions).flat_map(extract_context)


In [119]:
from functools import partial

generate_pairs = partial(generate_skipgram_pairs, window_size=2)
skipgram_ds = train_ds.flat_map(generate_pairs)

skipgram_ds

<_FlatMapDataset element_spec=(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [120]:
train_ds = (
    train_ds
    .map(
        lambda x: ((tf.one_hot(x[0], depth=vocab_size), tf.one_hot(x[1], depth=vocab_size)), tf.constant(1.0)),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
)

print(train_ds.element_spec)
next(train_ds.as_numpy_iterator())

((TensorSpec(shape=(8157,), dtype=tf.float32, name=None), TensorSpec(shape=(8157,), dtype=tf.float32, name=None)), TensorSpec(shape=(), dtype=tf.float32, name=None))


((array([0., 0., 1., ..., 0., 0., 0.], dtype=float32),
  array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)),
 np.float32(1.0))

In [123]:
tf.data.experimental.cardinality(train_ds).numpy()

np.int64(31014)

In [122]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dot, Activation

embedding_dims = [45, 312, 752]
window_sizes   = [2, 4]
histories      = {}
for w in window_sizes:
    for ed in embedding_dims:
        # One-hot inputs (shape = (vocab_size,))
        input_target = Input(shape=(vocab_size,))
        input_context = Input(shape=(vocab_size,))

        # Shared embedding layer simulated with Dense layer
        embedding_layer = Dense(ed, use_bias=False)

        target_embedding = embedding_layer(input_target)   # shape: (embedding_dim,)
        context_embedding = embedding_layer(input_context) # same

        # Dot product of embeddings
        dot_product = Dot(axes=-1)([target_embedding, context_embedding])

        # Sigmoid output for skipgram-style binary prediction
        output = Activation('sigmoid')(dot_product)

        model = Model(inputs=[input_target, input_context], outputs=output)
        model.compile(optimizer='adam', loss='binary_crossentropy')

        history = model.fit(train_ds.batch(128).prefetch(tf.data.AUTOTUNE), epochs=5)

        histories[(w, ed)] = history

list(histories.keys())

Epoch 1/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 0.6577
Epoch 2/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.3212
Epoch 3/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.1664
Epoch 4/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.1053
Epoch 5/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0723
Epoch 1/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 30s 124ms/step - loss: 0.5870
Epoch 2/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - loss: 0.1609
Epoch 3/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - loss: 0.0669
Epoch 4/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - loss: 0.0338
Epoch 5/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - loss: 0.0191
Epoch 1/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 68s 280ms/step - loss: 0.5400
Epoch 2/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 68s 280ms/step - loss: 0.1121
Epoch 3/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 66s 274ms/step - loss: 0.0383
Epoch 4/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 68s 279ms/step - loss: 0.0167
Epoch 5/5
243/243 ━━━━━━━━━━━━━━━━━━━━ 67s 2

[(2, 45), (2, 312), (2, 752), (4, 45), (4, 312), (4, 752)]

In [46]:
embeddings = embedding_layer.get_weights()[0]

print(embeddings.shape)

(8157, 128)


In [47]:
# prompt: given vectorize_layer how to obtain the index for the word "what"

import numpy as np

# Get the vocabulary from the vectorize_layer
vocabulary = vectorize_layer.get_vocabulary()

# Find the index of the word "what"
try:
    what_index = vocabulary.index("what")
    print(f"The index of 'what' in the vocabulary is: {what_index}")
except ValueError:
    print("'what' is not found in the vocabulary.")


The index of 'what' in the vocabulary is: 52


In [48]:
embeddings[what_index].shape

(128,)

In [49]:
print(train_ds.element_spec)
next(train_ds.as_numpy_iterator())

((TensorSpec(shape=(8157,), dtype=tf.float32, name=None), TensorSpec(shape=(8157,), dtype=tf.float32, name=None)), TensorSpec(shape=(), dtype=tf.float32, name=None))


((array([0., 0., 1., ..., 0., 0., 0.], dtype=float32),
  array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)),
 np.float32(1.0))

In [ ]:
classifier_train_ds = tf.data.Dataset.from_tensor_slices((vectorized_train, df_train["sentiment"].values))

print(classifier_train_ds.element_spec)
next(classifier_train_ds.as_numpy_iterator())

(TensorSpec(shape=(33,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))


(array([  293,    17, 15185,    69,     2,   120,    47,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0]),
 b'neutral')

In [ ]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=embeddings.shape[0],
    output_dim=embeddings.shape[1],
    weights=[embeddings],
    trainable=False
)

def embed_and_flatten(indices, label):
    embedded = embedding_layer(indices)        # shape: (seq_len, embed_dim)
    flat = tf.reshape(embedded, [-1])          # shape: (seq_len * embed_dim,)
    return flat, label

In [ ]:
classifier_train_ds = classifier_train_ds.map(embed_and_flatten)
print(classifier_train_ds.element_spec)
next(classifier_train_ds.as_numpy_iterator())

(TensorSpec(shape=(4224,), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))


(array([-0.17132318, -0.15126888, -0.17530611, ...,  0.4272256 ,
        -0.41871053, -0.40135565], dtype=float32),
 b'neutral')

In [ ]:
# Define a lookup table to convert string → integer
label_lookup = tf.keras.layers.StringLookup(
    vocabulary=df_train["sentiment"].unique().tolist(),
    num_oov_indices=0,
)

# Optional: one-hot encode
num_classes = label_lookup.vocabulary_size()

classifier_train_ds = classifier_train_ds.map(lambda x, y: (x, tf.one_hot(label_lookup(y), depth=num_classes)))

print(classifier_train_ds.element_spec)
next(classifier_train_ds.as_numpy_iterator())

(TensorSpec(shape=(4224,), dtype=tf.float32, name=None), TensorSpec(shape=(3,), dtype=tf.float32, name=None))


(array([-0.17132318, -0.15126888, -0.17530611, ...,  0.4272256 ,
        -0.41871053, -0.40135565], dtype=float32),
 array([1., 0., 0.], dtype=float32))

In [ ]:
classifier_train_ds = classifier_train_ds.batch(128)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3-class classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(classifier_train_ds, epochs=3)


Epoch 1/3
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.3925 - loss: 0.6410
Epoch 2/3
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4302 - loss: 0.6215
Epoch 3/3
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4364 - loss: 0.6138
